<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>


## Continuous integration report

Each notebook indicates its start and end of execution:
 - at the beginning, with `pwk.init()`
 - at the end, with `pwk.end()`  
 
All of theses informations are saved in a json file - Final report is below and in a HTML file.  
See ./log

In [1]:
import sys,os
import json
import datetime, time
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import re
import yaml
from collections import OrderedDict
from IPython.display import display
sys.path.append('..')
import fidle.config as config
import fidle.cooker as cooker

In [2]:

_start_time=None

def get_profile(existant_config=None, output_tag='==done==', save_figs=True):
    
    catalog = cooker.read_catalog()

    config   = {'version':'1.0', 'output_tag':output_tag, 'save_figs':save_figs}
    profile  = { 'config':config }
    for id, about in catalog.items():
        
        id        = about['id']
        title     = about['title']
        dirname   = about['dirname']
        basename  = about['basename']
        overrides = about.get('overrides',None)
    
        notebook = {}
        notebook['notebook_dir'] = dirname
        notebook['notebook_src'] = basename
        notebook['notebook_out'] = 'default'
        if len(overrides)>0:
            notebook['overrides']={ name:'default' for name in overrides }
                    
        profile[id]=notebook
        
    return profile


def save_profile(profile, filename):
    with open(filename,'wt') as fp:
        yaml.dump(profile, fp, sort_keys=False)
        print(f'Catalog saved as {filename}')
        print('Entries : ',len(profile)-1)

def load_profile(filename):
    with open(filename,'r') as fp:
        profile=yaml.load(fp, Loader=yaml.FullLoader)
        print(f'{filename} loaded.')
        print('Entries : ',len(profile)-1)
        return profile
    
def chrono_start():
    global _start_time
    _start_time = time.time()

def chrono_hdelay():
    global _start_time
    delay=int( time.time() - _start_time )
    return str(datetime.timedelta(seconds=delay))





def run_profile(profile, top_dir='..'):

    # ---- My place
    #
    home = os.getcwd()
    
    # ---- Read profile
    #
    config    = profile['config']
    notebooks = profile
    del notebooks['config']
    
    # ---- Save figs or not ?
    #
    os.environ['FIDLE_SAVE_FIGS']=str(config['save_figs'])

    # ---- For each notebook
    #
    for id,about in notebooks.items():
        
        print(f'\nNotebook : {id}')

        # ---- Get notebook infos ---------------------------------------------
        #
        notebook_dir = about['notebook_dir']
        notebook_src = about['notebook_src']
        notebook_out = about['notebook_out']
        overrides    = about.get('overrides',None)

        # ---- notebook_out (Default)
        #
        if notebook_out=='default':
            notebook_out = notebook_src.replace('.ipynb', config['output_tag']+'.ipynb')
                
        # ---- Override ------------------------------------------------------- 
        
        to_unset=[]
        if isinstance(overrides,dict):
            print('    set overrides :')
            for name,value in overrides.items():
                # ---- Default : no nothing
                if value=='default' : continue
                #  ---- Set env
                env_name  = f'FIDLE_OVERRIDE_{id.upper()}_{name}'
                env_value = str(value)
                os.environ[env_name] = env_value
                # ---- For cleaning
                to_unset.append(env_name)
                # ---- Fine :(-)
                print(f'       {env_name:38s} = {env_value}')
    
        # ---- Run it ! -------------------------------------------------------
    
        # ---- Go to the right place
        #
        os.chdir(f'{top_dir}/{notebook_dir}')
        notebook = nbformat.read( f'{notebook_src}', nbformat.NO_CONVERT)

        print('    Run notebook...',end='')
        ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
        chrono_start()
        ep.preprocess(notebook)
        print('..done.')
        print('    Duration : ',chrono_hdelay() )
    
        # ---- Save it
        #
        with open( f'{notebook_out}', mode="w", encoding='utf-8') as fp:
            nbformat.write(notebook, fp)
        print('    Saved as : ',notebook_out)
    
        # ---- Back to home and clean
        #
        os.chdir(home)
        for env_name in to_unset:
            del os.environ[env_name] 
    
    
# ---- Save profile
    
# profile = get_profile()
# save_profile(profile, './ci/default.yml')
# profile=load_profile('./ci/default.yml')
    
    
# ---- Run profile    
    
profile = load_profile('./ci/smart.yml')

run_profile(profile)



./ci/smart.yml loaded.
Entries :  4

Notebook : LINR1
    Run notebook.....done.
    Duration :  0:00:04
    Saved as :  01-Linear-Regression==done==.ipynb

Notebook : GTSRB0
    set overrides :
       FIDLE_OVERRIDE_GTSRB0_scale            = 0.7
       FIDLE_OVERRIDE_GTSRB0_output_dir       = ./data/foobar
       FIDLE_OVERRIDE_GTSRB0_run_dir          = ./run/xxx
    Run notebook.....done.
    Duration :  0:00:02
    Saved as :  00-Test==done==.ipynb

Notebook : GTSRB1
    set overrides :
       FIDLE_OVERRIDE_GTSRB1_scale            = 0.1
       FIDLE_OVERRIDE_GTSRB1_output_dir       = ./data
    Run notebook.....done.
    Duration :  0:02:00
    Saved as :  01-Preparation-of-data==done==.ipynb

Notebook : GTSRB2
    set overrides :
       FIDLE_OVERRIDE_GTSRB2_run_dir          = ./run/GTSRB2_ci
       FIDLE_OVERRIDE_GTSRB2_dataset_name     = set-24x24-L
       FIDLE_OVERRIDE_GTSRB2_batch_size       = 32
       FIDLE_OVERRIDE_GTSRB2_epochs           = 3
       FIDLE_OVERRIDE_GTSRB2_s

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>